Machine Learning I exercise done by Cedric Prieels.

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Using convnets with small datasets

The first thing to do is download the following file https://lara.web.cern.ch/lara/train.zip in the jupyter terminal and uncompress it in the same folder as this notebook. 

To download another dataset from imagenet you can do it with the URL list of the images and using `wget -i`



## Training a convnet from scratch

Training an image classification model with very little data is a common situation you will find yourself in if you end up doing Computer Vision in a professional context.  

Having "few" samples can mean anything from a few hundred to a few tens of thousands of images.  Let's illustrate a practical example here: let's focus on classifying images as "dogs" and "cats". 


## The importance of DEEP Learning in problems with few data

You may have heard many times that Deep Learning only works when you have large amounts of data. This is partly true: one of the characteristics of Deep learning is that you can find interesting features from the training dataset itself, and this a priori is easier when you have many examples available, especially in the case of having input datasets with a high dimensionality, such as images.

However, what constitutes a "large" dataset is relative. Specifically relative to the size and depth of the network we are trying to train. It is not possible to sand a convnet so that it becomes a complete problem with only a few dozen examples, but a few hundred can be enough if the model is well assembled (we will understand what "well assembled" means throughout the Deep Learning course).

Since convnets learn local characteristics, invariant under translations, they are very efficient in terms of the number of images needed to carry out perceptual problems. So training a convnet from 0 with a not very large dataset can still lead to reasonable results as we will see here.

But there is more: Deep Learning models are highly "recyclable". One can, for example, take an image classification problem and a trained speech-to-text converter on a very big dataset and then reuse it for solving a completely different problem only by adding some small modifications. More specifically, in the case of Computer Vision, many pre-trained models (usually trained on the ImageNet dataset) are made public so that one can download them and use them to create powerful Computer Vision models with very little data.

But here we will just run a simple example.
 


## Los datos

The cat vs dog dataset we use is not a Keras package. It was posted on Kaggle.com as part of a Computer Vision problem in late 2013, when ConvNets were not yet so popular. 

The images are medium resolution JPGEs. It looks like this:

![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

It's no surprise that the 2013 Kaggle cat vs dog competition was won by ConvNets. The best were able to achieve up to 95% accuracy. In our example we are still far from this accuracy, but during the Deep Learning course we have learned how to approach this value using different methods to improve the performance of neural networks. It should be noted that in this example we are training on approximately only 10% of the data that was used for the contest. 
After downloading the dataset and decompressing it, we are going to create a new dataset containing three subsets: a training set containing 1000 images of each class, a validation set with 500 images of each class, and finally a test set with 500 images of each class.

Here we have a few lines of code that make us this distribution automatically:




In [2]:
import os, shutil

In [4]:
# The path to the directory where the original
# dataset was uncompressed
original_dataset_dir = 'train'

# The directory where we will
# store our smaller dataset
base_dir = 'cats_and_dogs_small'
os.mkdir(base_dir)

# Directories for our training,
# validation and test splits
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

# Directory with our validation cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

# Directory with our validation dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Copy first 1000 cat images to train_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

As a sanity check, let's count how many pictures we have in each training split (train/validation/test):

In [5]:
print('total training cat images:', len(os.listdir(train_cats_dir)))

total training cat images: 1000


In [6]:
print('total training dog images:', len(os.listdir(train_dogs_dir)))

total training dog images: 1000


In [7]:
print('total validation cat images:', len(os.listdir(validation_cats_dir)))

total validation cat images: 500


In [8]:
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

total validation dog images: 500


In [9]:
print('total test cat images:', len(os.listdir(test_cats_dir)))

total test cat images: 500


In [10]:
print('total test dog images:', len(os.listdir(test_dogs_dir)))

total test dog images: 500


So effectively we have 2000 training images, 1000 validation images and 1000 test images. In each of these subsets there are the same number of examples from each class: this is what is called a balanced binary classification system, which means that our classification accuracy will be an adequate metric of the success of our solution.

## Building our network

In the above example we have built a small convnet to solve the problem of classifying handwritten numbers using the MNIST dataset, so we are already familiar with the terminology that keras uses. We are going to reuse the general structure we had in the previous example: our convnet will have a stack of alternate layers of `Conv2D` (with `relu` activation) and `MaxPooling2D` layers.

However, since we are dealing with larger images and a more complex problem, we will create our network accordingly: it will have one more layer of `Conv2D` + `MaxPooling2D`. This serves to increase the capacity of the network and to further reduce the size of the feature maps, so that they are not so huge when they reach the flattening step. We start using 150x150 input images (an arbitrary choice), and end up with feature maps that are 7x7 in size before the flattening layer.

It is important to note that the depth of feature maps progressively increases as we move through the neural network (from 32 to 128) while the size of feature maps decreases (from 148x148 to 7x7). You will see this pattern in almost all convnets.

As we are attacking a binary classification problem (dog or cat), we are going to finish the network with a single unit (a dense layer of size 1) and with a sigmoid activation. This unit will encode the probability that our network is looking at one class or another.

The final look of the model should be as follows:


![modelo_red_animales.png](https://github.com/laramaktub/MachineLearningI/blob/master/modelo_red_animales.png?raw=true)

In [11]:
from keras import layers
from keras import models
from keras.layers import Dense, Activation, Flatten

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))) #T150x150px dataset input, 3 colors
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='linear'))
model.add(layers.MaxPooling2D((2, 2)))

#Next step
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary() #Check the results, we now have more than 3 million parameters!

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

Para el paso de compilación utilizaremos el optimizador `RMSprop`(lr=1e-4). Como nuestra red termina con una única unidad sigmoide, vamos a utilizar binary crossentropy como nuestra función de pérdida.

In [12]:
from keras import optimizers

model.compile(optimizer=optimizers.rmsprop(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'])

##  Data preprocessing

The images must be properly formatted as float tensors before they are given to the net. That's just what we're going to do here. Before we pre-process them, the images are JPEG files. The steps to be able to give them to our network are roughly as follows:

* Read the files with the images.
* Decode the content of the JPEG in a "grid" with the RGB of the pixels 
* Turn that "grill" into floatation devices
* Rescale the pixel values (between 0 and 255) to the [0, 1] interval as neural networks prefer to work with small values. 

All this may seem very complicated but thanks to Keras our life is much easier and we can count on your tools to take care of these steps automatically. Keras has a module with tools for image processing, which can be found in `keras.preprocessing.image`. In particular, it contains the class `ImageDataGenerator` that allows us to automatically convert images we have on the hard disk into pre-processed tensors. This is exactly what we'll be using next. To do this we can use the flow_from_directory to take the images directly from the folders that we previously generated. We give it as input the folders where the training (or validation) images are, the size of the images (target_size), the size of the batch we're going to use (we're going to start with 20) and as there are only two categories, we tell it that we're going to use binary_crossentropy (class_mode). When we run these commands we'll get the following, the total number of images and how many classes there are.

In [13]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(directory="cats_and_dogs_small/train/", target_size=(150, 150), batch_size=20, class_mode='binary')
test_generator = test_datagen.flow_from_directory(directory="cats_and_dogs_small/test/", target_size=(150, 150), batch_size=20, class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(directory="cats_and_dogs_small/validation", target_size=(150, 150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


Let's take a look at one of these generators: it takes us to a batch of 150x150 RGB images (dimensions `(20, 150, 150, 3)`) and binary tags (dimension `(20,)`). 20 is the number of examples in each batch (what we call the batch size). The generator generates these batches indefinitely: it runs a loop endlessly through all the images we have in the folder. That's why we have to type `break` to break the loop at some point.

In [14]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


Now let's make the fit. In this case, as what we have is a generator, we use fit_generator. We are going to run 30 epochs and use the validation dataset.

In [15]:
history = model.fit_generator(train_generator, epochs=30, validation_data=validation_generator)

Epoch 1/30
100/100 [==============================] - 83s 827ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 2/30
100/100 [==============================] - 83s 831ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 3/30
100/100 [==============================] - 82s 817ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 4/30
100/100 [==============================] - 81s 814ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 5/30
100/100 [==============================] - 86s 860ms/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 6/30
100/100 [==============================] - 121s 1s/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 7/30
100/100 [==============================] - 122s 1s/step - loss: 7.9712 - acc: 0.5000 - val_loss: 7.9712 - val_acc: 0.5000
Epoch 8/30
100/100 [==============================] - 121s 

It's a nice idea to save the model after training

In [16]:
model.save('net_numbers.h5')

Let's now evaluate our model using the test dataset

In [17]:
scores = model.evaluate_generator(test_generator)
print("Loss = ", scores[0])
print("Accuracy = ", scores[1])

Loss =  7.9711928558349605
Accuracy =  0.5000000014901161


Try to optimize the network with the tools you have learnt during the lesson. Try to make improvements both in terms of speed and accuracy. Comment the results.

As we saw previously, the model that we considered was far from great since its accuracy was low (only 50% for two categories, the same as guessing) and it was that it was not able to learn, since the accuracy and the loss stayed constant. For this exercise, in order to reduce the number of hyperparameters to optimize, we will not change the model previously defined by adding/removing layers/neurons, we are just going to play with the fitting and training parameters (mainly the learning rate of the optimizer, the rescale parameter, the batch size and the parameters related to the fit itself). Let's also make this process a bit more automatic by defining a function.

In [24]:
def reset_weights(model):
    import keras.backend as K
    session = K.get_session()
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'): 
            layer.kernel.initializer.run(session=session)
        if hasattr(layer, 'bias_initializer'):
            layer.bias.initializer.run(session=session)  

def trainAndEvaluate(model, learningRate = 1e-4, rescale = False, batchSize = 20, epochs = 30, name="myModel"):
    #First, reset the weights of the model
    reset_weights(model)
    
    model.compile(optimizer=optimizers.rmsprop(lr=1e-4), loss="binary_crossentropy", metrics=['accuracy'])

    if rescale:
        train_datagen = ImageDataGenerator(rescale=1. / 255)
        test_datagen = ImageDataGenerator(rescale=1. / 255)
        validation_datagen = ImageDataGenerator(rescale=1. / 255)
    else:
        train_datagen = ImageDataGenerator()
        test_datagen = ImageDataGenerator()
        validation_datagen = ImageDataGenerator()
        
    train_generator = train_datagen.flow_from_directory(directory="cats_and_dogs_small/train/", target_size=(150, 150), batch_size=batchSize, class_mode='binary')
    test_generator = test_datagen.flow_from_directory(directory="cats_and_dogs_small/test/", target_size=(150, 150), batch_size=batchSize, class_mode='binary')
    validation_generator = validation_datagen.flow_from_directory(directory="cats_and_dogs_small/validation", target_size=(150, 150), batch_size=batchSize, class_mode='binary')
    
    history = model.fit_generator(train_generator, epochs=epochs, validation_data=validation_generator)
    model.save(str(name)+'.h5')
    
    scores = model.evaluate_generator(test_generator)
    print("Loss = ", scores[0])
    print("Accuracy = ", scores[1])
    return scores

First, let's try by simply using the rescale parameters of the ImageDataGenerator(), since it seems that it does improve a lot the results obtained. Now, our model is able to learn but we can clearly see that a constant accuracy with the validation dataset is already obtain after around 10 epochs, so let's use this value from now on for this exercise in order to speed up a bit the training and avoid overtraining.

In [25]:
trainAndEvaluate(model, learningRate = 1e-4, rescale = True, batchSize = 20, epochs = 30, name="myModel")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/30
100/100 [==============================] - 83s 835ms/step - loss: 0.6875 - acc: 0.5430 - val_loss: 0.6646 - val_acc: 0.5910
Epoch 2/30
100/100 [==============================] - 81s 812ms/step - loss: 0.6412 - acc: 0.6265 - val_loss: 0.6213 - val_acc: 0.6540
Epoch 3/30
100/100 [==============================] - 81s 815ms/step - loss: 0.5813 - acc: 0.6905 - val_loss: 0.6114 - val_acc: 0.6510
Epoch 4/30
100/100 [==============================] - 81s 812ms/step - loss: 0.5389 - acc: 0.7220 - val_loss: 0.5719 - val_acc: 0.6960
Epoch 5/30
100/100 [==============================] - 81s 811ms/step - loss: 0.5049 - acc: 0.7460 - val_loss: 0.5761 - val_acc: 0.6980
Epoch 6/30
100/100 [==============================] - 82s 819ms/step - loss: 0.4721 - acc: 0.7775 - val_loss: 0.5741 - val_acc: 0.7010
Epoch 7/30
100/100 [==============================] - 83s 828ms/

[1.5449113744497298, 0.7090000009536743]

Let's now try to change the learning rate to see if it changes anything. It does not seem to have much of an impact anyway.

In [26]:
trainAndEvaluate(model, learningRate = 1e-5, rescale = True, batchSize = 20, epochs = 10, name="myModel2")
trainAndEvaluate(model, learningRate = 1e-3, rescale = True, batchSize = 20, epochs = 10, name="myModel3")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
100/100 [==============================] - 89s 887ms/step - loss: 0.6864 - acc: 0.5515 - val_loss: 0.6616 - val_acc: 0.6140
Epoch 2/10
100/100 [==============================] - 92s 919ms/step - loss: 0.6410 - acc: 0.6265 - val_loss: 0.6216 - val_acc: 0.6590
Epoch 3/10
100/100 [==============================] - 90s 895ms/step - loss: 0.5776 - acc: 0.6905 - val_loss: 0.5998 - val_acc: 0.6730
Epoch 4/10
100/100 [==============================] - 86s 857ms/step - loss: 0.5369 - acc: 0.7215 - val_loss: 0.5942 - val_acc: 0.6900
Epoch 5/10
100/100 [==============================] - 85s 847ms/step - loss: 0.5013 - acc: 0.7540 - val_loss: 0.6038 - val_acc: 0.6780
Epoch 6/10
100/100 [==============================] - 87s 869ms/step - loss: 0.4731 - acc: 0.7740 - val_loss: 0.5771 - val_acc: 0.7010
Epoch 7/10
100/100 [==============================] - 87s 874ms/

[0.5863171517848969, 0.7120000016689301]

We can also try to increase/decrease the batchSize. A lower batchsize increase by ~20% the training time and does not improve the accuracy. A batch size of 50, on the other hand, lowers the training time by ~10% while keeping a similar accuracy (even though slightly smaller).

In [30]:
trainAndEvaluate(model, learningRate = 1e-4, rescale = True, batchSize = 10, epochs = 10, name="myModel4")
trainAndEvaluate(model, learningRate = 1e-4, rescale = True, batchSize = 50, epochs = 10, name="myModel5")

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Epoch 1/10
200/200 [==============================] - 97s 487ms/step - loss: 0.6837 - acc: 0.5555 - val_loss: 0.6745 - val_acc: 0.5670
Epoch 2/10
200/200 [==============================] - 96s 482ms/step - loss: 0.6147 - acc: 0.6635 - val_loss: 0.6065 - val_acc: 0.6630
Epoch 3/10
200/200 [==============================] - 96s 478ms/step - loss: 0.5502 - acc: 0.7200 - val_loss: 0.6071 - val_acc: 0.6670
Epoch 4/10
200/200 [==============================] - 96s 481ms/step - loss: 0.4971 - acc: 0.7460 - val_loss: 0.6140 - val_acc: 0.6830
Epoch 5/10
200/200 [==============================] - 97s 484ms/step - loss: 0.4472 - acc: 0.7940 - val_loss: 0.5780 - val_acc: 0.7050
Epoch 6/10
200/200 [==============================] - 97s 483ms/step - loss: 0.3906 - acc: 0.8250 - val_loss: 0.6028 - val_acc: 0.7060
Epoch 7/10
200/200 [==============================] - 97s 483ms/

[0.5659809440374375, 0.6949999958276749]

The best parameters obtained for the model defined in this case are 10 training epochs, a learning rate of 1e-4, rescaling the data and a batchsize of 50.